In [20]:
import sentencepiece as spm
import os

In [21]:
## Here, you can see how many lines each file has

!wc -l ./raw_data/train.fa
!wc -l ./raw_data/train.en
!wc -l ./raw_data/valid.fa
!wc -l ./raw_data/valid.en
!wc -l ./raw_data/test.fa
!wc -l ./raw_data/test.fa

670000 ./raw_data/train.fa
670000 ./raw_data/train.en
4000 ./raw_data/valid.fa
4000 ./raw_data/valid.en
10000 ./raw_data/test.fa
10000 ./raw_data/test.fa


In [22]:
!head ./raw_data/train.fa -n 3
!head ./raw_data/train.en -n 3

مرگ 50 ستیزه جوی دیگر در عملیات وزیرستان شمالی
پیشاور - به گزارش جیو نیوز , عملیات ادامه دار ارتش در وزیرستان شمالیمنجر به مرگ دست کم 50 ستیزه جوی مظنون دیگر در روز 17 ژوئن شد .
به گزارش رسانه‌ها , نیروی هوایی پاکستان حملات هوایی را علیه مخفیگاه های ستیزه جویی در میرعلی و نواحی دیگر منطقه انجام داد و دست کم هشت مخفیگاه را ویران کرد .
North Waziristan operation kills 50 more militants
PESHAWAR - The on - goingmilitary operation in North Waziristankilled at least 50 more suspected militants June 17 , Geo News reported .
The Pakistani Air Force conducted airstrikes against militant hideouts in Mir Ali and other areas of the agency , destroying at least eight , media reported .


In [23]:
#join all paths to a base path 
BASE_PATH = "."
RAW_DATA_PATH = os.path.join(BASE_PATH, "raw_data")
TOKENIZED_DATA_PATH = os.path.join(BASE_PATH, "tokenized_data")
VOCAB_SIZE = 10000

In [24]:
# Do this only if  m_fa is not created in ./tokenizer

INPUT_TRAIN_FA = os.path.join(RAW_DATA_PATH, "train.fa")
OUTPUT_MODEL_FA = os.path.join(TOKENIZED_DATA_PATH, "m_fa.model")

spm.SentencePieceTrainer.train(input=INPUT_TRAIN_FA, model_prefix=OUTPUT_MODEL_FA, vocab_size=VOCAB_SIZE, character_coverage=1.0, model_type='bpe')


KeyboardInterrupt: 

In [ ]:
INPUT_TRAIN_EN = os.path.join(RAW_DATA_PATH, "train.en")
OUTPUT_MODEL_EN = os.path.join(TOKENIZED_DATA_PATH, "m_en.model")


spm.SentencePieceTrainer.train(input=INPUT_TRAIN_EN, model_prefix=OUTPUT_MODEL_EN, vocab_size=VOCAB_SIZE, character_coverage=1.0, model_type='bpe')


the model already exists


In [ ]:
# Read all raw files as lists of strings by line break
with open(os.path.join(RAW_DATA_PATH, "train.fa"), "r") as f:
    train_fa = f.readlines()
with open(os.path.join(RAW_DATA_PATH, "train.en"), "r") as f:
    train_en = f.readlines()
with open(os.path.join(RAW_DATA_PATH, "valid.fa"), "r") as f:
    valid_fa = f.readlines()
with open(os.path.join(RAW_DATA_PATH, "valid.en"), "r") as f:
    valid_en = f.readlines()
with open(os.path.join(RAW_DATA_PATH, "test.fa"), "r") as f:
    test_fa = f.readlines()
with open(os.path.join(RAW_DATA_PATH, "test.en"), "r") as f:
    test_en = f.readlines()
    
print(f"the length of train_fa is {len(train_fa)} and the length of train_en is {len(train_en)}")


the length of train_fa is 670000 and the length of train_en is 670000


In [ ]:
train_fa[0]

'مرگ 50 ستیزه جوی دیگر در عملیات وزیرستان شمالی'

In [ ]:
train_en[0]

'North Waziristan operation kills 50 more militants'

In [ ]:
sp_fa = spm.SentencePieceProcessor(model_file=OUTPUT_MODEL_FA)
sp_en = spm.SentencePieceProcessor(model_file=OUTPUT_MODEL_EN)

In [ ]:
encoded_train_fa = sp_fa.encode(train_fa, out_type=str)
encoded_train_en = sp_en.encode(train_en, out_type=str)
encoded_valid_fa = sp_fa.encode(valid_fa, out_type=str)
encoded_valid_en = sp_en.encode(valid_en, out_type=str)
encoded_test_fa = sp_fa.encode(test_fa, out_type=str)
encoded_test_en = sp_en.encode(test_en, out_type=str)

encoded_train_fa[0]


NameError: name 'encoded_fa' is not defined

In [ ]:
encoded_train_fa[0]


['▁مرگ',
 '▁50',
 '▁ستیزه',
 '▁جوی',
 '▁دیگر',
 '▁در',
 '▁عملیات',
 '▁وزیرستان',
 '▁شمالی']

In [ ]:
# turn each list of strings into a single string with space as the delimiter
encoded_train_fa_str = [' '.join(sent) for sent in encoded_train_fa]
encoded_train_en_str = [' '.join(sent) for sent in encoded_train_en]
encoded_valid_fa_str = [' '.join(sent) for sent in encoded_valid_fa]
encoded_valid_en_str = [' '.join(sent) for sent in encoded_valid_en]
encoded_test_fa_str = [' '.join(sent) for sent in encoded_test_fa]
encoded_test_en_str = [' '.join(sent) for sent in encoded_test_en]

encoded_train_fa_str[0]

'▁مرگ ▁50 ▁ستیزه ▁جوی ▁دیگر ▁در ▁عملیات ▁وزیرستان ▁شمالی'

In [ ]:
# write all arrays of last cell to files in ./tokenized_data

with open(os.path.join(TOKENIZED_DATA_PATH, "train.fa"), "w") as f:
    f.writelines(encoded_train_fa_str)
with open(os.path.join(TOKENIZED_DATA_PATH, "train.en"), "w") as f:
    f.writelines(encoded_train_en_str)
with open(os.path.join(TOKENIZED_DATA_PATH, "valid.fa"), "w") as f:
    f.writelines(encoded_valid_fa_str)
with open(os.path.join(TOKENIZED_DATA_PATH, "valid.en"), "w") as f:
    f.writelines(encoded_valid_en_str)
with open(os.path.join(TOKENIZED_DATA_PATH, "test.fa"), "w") as f:
    f.writelines(encoded_test_fa_str)
with open(os.path.join(TOKENIZED_DATA_PATH, "test.en"), "w") as f:
    f.writelines(encoded_test_en_str)



In [ ]:
show_tokenized_top_fa_command = f"head {TOKENIZED_DATA_PATH}/train.fa -n 5"
show_tokenized_top_en_command = f"head {TOKENIZED_DATA_PATH}/train.en -n 5"
!{show_tokenized_top_en_command}
!{show_tokenized_top_fa_command}

▁North ▁Waziristan ▁operation ▁kills ▁50 ▁more ▁militants
▁PESHAWAR ▁- ▁The ▁on ▁- ▁going m ilitary ▁operation ▁in ▁North ▁Waziristan k illed ▁at ▁least ▁50 ▁more ▁suspected ▁militants ▁June ▁17 ▁, ▁Geo ▁News ▁reported ▁.
▁The ▁Pakistani ▁Air ▁Force ▁conducted ▁air st rik es ▁against ▁militant ▁hideouts ▁in ▁Mir ▁Ali ▁and ▁other ▁areas ▁of ▁the ▁agency ▁, ▁destroying ▁at ▁least ▁eight ▁, ▁media ▁reported ▁.
▁A ▁day ▁earlier ▁, ▁at ▁least ▁27 ▁militants ▁were ▁reported ▁dead ▁after ▁j ets ▁p ounded ▁militant ▁hideouts ▁in ▁Sh aw al ▁.
▁Meanwhile ▁, ▁a ▁roadside ▁blast ▁on ▁Bang id ar ▁road ▁in ▁the ▁Gh ul lam ▁Khan ▁area ▁June ▁16 ▁killed ▁six ▁security ▁personnel ▁and ▁wounded ▁three ▁others ▁, ▁according ▁to ▁officials ▁.
▁مرگ ▁50 ▁ستیزه ▁جوی ▁دیگر ▁در ▁عملیات ▁وزیرستان ▁شمالی
▁پیشاور ▁- ▁به ▁گزارش ▁جیو ▁نیوز ▁, ▁عملیات ▁ادامه ▁دار ▁ارتش ▁در ▁وزیرستان ▁شمال یمن جر ▁به ▁مرگ ▁دست ▁کم ▁50 ▁ستیزه ▁جوی ▁مظنون ▁دیگر ▁در ▁روز ▁17 ▁ژوئن ▁شد ▁.
▁به ▁گزارش ▁رسانه ▁ها ▁, ▁نیروی ▁هوایی ▁پاکستان ▁